In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# other utilities
from sklearn import datasets, preprocessing, metrics

In [50]:
# read the Excel file from your data folder into a data frame
df = pd.read_excel('../data/SOFT-3.xlsx', index_col=None, na_values=['NA'])

In [51]:
# see the size
df.shape

(20, 3)

In [4]:
df.columns

Index(['SOFT Internship 2023', 'Unnamed: 1', 'Unnamed: 2'], dtype='object')

In [5]:
# see which are the attribute labels
list(df)

['SOFT Internship 2023', 'Unnamed: 1', 'Unnamed: 2']

In [6]:
# see the first five records
df.head()

,SOFT Internship 2023,Unnamed: 1,Unnamed: 2
0,NaN,student,company
1,NaN,Benjamin Curovic,Dynatest A/S
2,NaN,Jacob Simonsen,"Eriksholm Research Centre, Oticon"
3,NaN,Sebastian Godsk Hansen,Formpipe
4,NaN,Lukas Bang Stoltz-Andersen,Formpipe


In [7]:
del df['SOFT Internship 2023']
df = df.iloc[1:]
df.rename(columns = {'Unnamed: 1':'student', 'Unnamed: 2':'company'}, inplace = True)
df.head()

,student,company
1,Benjamin Curovic,Dynatest A/S
2,Jacob Simonsen,"Eriksholm Research Centre, Oticon"
3,Sebastian Godsk Hansen,Formpipe
4,Lukas Bang Stoltz-Andersen,Formpipe
5,Christoffer Ikizek Wegner,Formpipe


In [10]:

from faker import Faker
faker = Faker()# Test fake data generation
print("The Faker library can generate fake names. By running 'faker.name()', we get:")
faker.name()

Faker.seed(4321)
dict_names = {name: faker.name() for name in df['student'].unique()}
df['student'] = df['student'].map(dict_names)
df.head(10)

The Faker library can generate fake names. By running 'faker.name()', we get:


,student,company
1,Jason Brown,Dynatest A/S
2,Jacob Stein,"Eriksholm Research Centre, Oticon"
3,Cody Brown,Formpipe
4,Larry Morales,Formpipe
5,Jessica Hendricks,Formpipe
6,Brian Moore,Novo Nordisk
7,Scott Baker,PFA
8,Ruth Hoffman,Topdanmark
9,Daniel George,KMD A/S
10,David Moody,KMD A/S


In [57]:
#import requests
#
#cvr_number = 25678990
#
#url = f"https://cvrapi.dk/api?search={cvr_number}&country=dk"
#
#r = response = requests.get(url)
#
#if response.status_code == 200:
#    data = response.json()
#    print(data)
#else:
#    print("Failed to fetch data")

In [59]:
def fetch_company_data_from_csv(filename):   
    df_list = []
    with open(filename, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        # Skip the header row if it exists
        next(csv_reader, None)
        for row in csv_reader:
            name = row[2]
            url = f"https://cvrapi.dk/api?search={name}&country=dk"
            response = requests.get(url)
            data = response.json()
            if response.status_code == 200:
                new_row = pd.DataFrame(data, columns=["name", "address", "zipcode", "city", "phone"], index=[1])
                df_list.append(new_row)
    df = pd.concat(df_list, ignore_index=True)
    return df

In [54]:
df = fetch_company_data_from_csv("SOFT-3.csv") 

In [2]:
df.head()

In [1]:
new_df = pd.DataFrame()

# pgeocode object
country = pgeocode.Nominatim('dk')

# Looper gennem hver enkelt "zipcode" value i dataframet
for index, row in df.iterrows():
    # bruger pgeocode til at få  information for den enkelte zipcode
    info = country.query_postal_code(row['zipcode'])
    # tilføjer informationen til det nye DataFrame
    new_df = pd.concat([new_df, info.to_frame().T], ignore_index=True)

# kombinere det nye med det gamle dataframe
combined_df = pd.concat([df, new_df], axis=1)

In [ ]:
#Concatter mit df med fake navne med de to dataframes jeg lige har sat sammen ovenover
result = pd.concat([df, combined_df], ignore_index=True)

In [ ]:
import smtplib
import ssl
from email.message import EmailMessage



email_sender = 'markus,agnsgaard@gmail.com'
email_password = '*********'
email_receiver = 'whoever the receiver is'

subject = 'Check out my new video!'
body = """
I've just published a new video on YouTube: https://youtu.be/2cZzP9DLlkg
"""

em = EmailMessage()
em['From'] = email_sender
em['To'] = email_receiver
em['Subject'] = subject
em.set_content(body)

context = ssl.create_default_context()


with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
    smtp.login(email_sender, email_password)
    smtp.sendmail(email_sender, email_receiver, em.as_string())